In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import collections, math, random, sys

In [ ]:
import torch 
torch.set_default_device('cuda')
import torch.nn as nn
import torch.optim as optim

In [ ]:
sys.path.append('/kaggle/input/nus-sms-corpus/')
print(sys.path)
from utils import *
print(sys.path)

In [ ]:
traindata = read_mono('/kaggle/input/nus-sms-corpus/data/small', delim='')
devdata = read_mono('/kaggle/input/nus-sms-corpus/data/dev', delim='')
testdata = read_mono('/kaggle/input/nus-sms-corpus/data/test', delim='')

vocab = Vocab()
for words in traindata:
    vocab |= words

In [ ]:
# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, voc_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.voc_dim = voc_dim
        self.lstm = nn.LSTMCell(voc_dim, hidden_dim)
        self.linear = nn.Linear(hidden_size, voc_dim)
        
    
    def forward(self, x):
        h_t = torch.zeros(x.size(0), self.hidden_dim)
        c_t = torch.zeros(x.size(0), self.hidden_dim)
        
        outputs = []
        
        for input_t in x:
            h_t, c_t = self.lstm(input_t, (h_t, c_t))
            output_t = self.linear(h_t)
            outputs.append(output_t)
            
        y_t = torch.log_softmax(outputs)
        
    
        